# Plot of posterior samples

_Kara Ponder (SLAC-->?), Emille Ishida (Clermont-Ferrand), Alex Malz (GCCL@RUB)_

plagiarized from `Combination_plots.ipynb`

In [ ]:
from collections import OrderedDict
import glob
import gzip
import numpy as np
import os
import pandas as pd
import pickle as pkl
import scipy.stats as sps
from matplotlib.ticker import MaxNLocator
import sys

## Kara's plotting code

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable

# import resspect.cosmo_metric_utils as cmu

In [ ]:
import pylab
from mpl_toolkits.axes_grid1 import ImageGrid
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


In [ ]:
all_shapes = {'SNIa-91bg': 'o',
              'SNIax': 's',
              'SNII': 'd',
              'SNIbc': 'X',
              'SLSN-I': 'v',
              'AGN': '^',
              'TDE': '<',
              'KN': '>',
              'CART': 'v'}

In [ ]:
# Color map
rainbow = cm = plt.get_cmap('plasma_r')
cNorm  = colors.LogNorm(vmin=1, vmax=52) #colors.Normalize(vmin=0, vmax=50)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=rainbow)
color_map = scalarMap.to_rgba(np.arange(1, 52))

## prep for data

In [ ]:
# DDF summary on the COIN server:
file_extensions = {'ddf': 'DDF', 
                   'wfd': 'WFD'
                  }
ktot = 5
kglob = ''
nobjs = '3000'

In [ ]:
def get_cases(field, k='', nobjs=3000):
    if k == '':
        k = '0'
    dirname = '/media/RESSPECT/data/PLAsTiCC/for_metrics/final_data3/'+field+'/results/v'+k+'/' + str(nobjs) + '/samples/'
    cases = os.listdir(dirname)
    #cases.remove('random1500.csv')
    #cases.remove('random6000.csv')
    #cases.remove('fiducial1500.csv')
    #cases.remove('fiducial6000.csv')
    #cases.remove('perfect6000.csv')
    #cases.remove('perfect1500.csv')
    
    if '.ipynb_checkpoints' in cases:
        cases.remove('.ipynb_checkpoints')
        
    #print(cases)
    return(cases, dirname)

In [ ]:
cases, dirnames = {}, {}
for file_extension in file_extensions:
    cases[file_extension], dirnames[file_extension] = get_cases(file_extensions[file_extension])

In [ ]:
def make_remap_dict(file_extension):
    if 'wfd' in file_extension:
        remap_dict = OrderedDict({
                              'perfect3000': 'Perfect', 
                              'fiducial3000': 'Fiducial', 
                              'random3000': 'Random',
                              #'all_objs_survived_SALT2_DDF' : 'All SALT',
                              #'all_objs_survived_SALT2_WFD': 'All SALT',
                              #'50SNIa50SNII': 'SN-II 50',
                              #'68SNIa32SNII': 'SN-II 32',
                              '72SNIa28SNII': 'SN-II 28',
                              '75SNIa25SNII': 'SN-II 25', 
                              '90SNIa10SNII': 'SN-II 10',
                              '95SNIa5SNII': 'SN-II 5',
                              '98SNIa2SNII': 'SN-II 2',
                              '99SNIa1SNII': 'SN-II 1',
                              #'50SNIa50SNIbc': 'SN-Ibc 50',
                              #'68SNIa32SNIbc': 'SN-Ibc 32',
                              #'75SNIa25SNIbc': 'SN-Ibc 25',
                              #'83SNIa17SNIbc': 'SN-Ibc 17',
                              '90SNIa10SNIbc': 'SN-Ibc 10',
                              '95SNIa5SNIbc': 'SN-Ibc 5',
                              '98SNIa2SNIbc': 'SN-Ibc 2',
                              '99SNIa1SNIbc': 'SN-Ibc 1',
                              #'50SNIa50SNIax': 'SN-Iax 50',
                              #'68SNIa32SNIax': 'SN-Iax 32',
                              '75SNIa25SNIax': 'SN-Iax 25',
                              #'86SNIa14SNIax': 'SN-Iax 14',
                              '90SNIa10SNIax': 'SN-Iax 10',
                              #'94SNIa6SNIax': 'SN-Iax 6',
                              '95SNIa5SNIax': 'SN-Iax 5',
                              #'97SNIa3SNIax': 'SN-Iax 3',
                              '98SNIa2SNIax': 'SN-Iax 2',
                              '99SNIa1SNIax': 'SN-Iax 1',
                              #'71SNIa29SNIa-91bg': 'SN-Ia-91bg 29',
                              #'75SNIa25SNIa-91bg': 'SN-Ia-91bg 25',
                              #'90SNIa10SNIa-91bg': 'SN-Ia-91bg 10',
                              '95SNIa5SNIa-91bg': 'SN-Ia-91bg 5',
                              '98SNIa2SNIa-91bg': 'SN-Ia-91bg 2',
                              '99SNIa1SNIa-91bg': 'SN-Ia-91bg 1',
                              #'99.8SNIa0.2SNIa-91bg': 'SN-Ia-91bg 0.2',
                              #'57SNIa43AGN': 'AGN 43',
                              #'75SNIa25AGN': 'AGN 25',
                              #'90SNIa10AGN': 'AGN 10',
                              #'94SNIa6AGN': 'AGN 6',
                              #'95SNIa5AGN': 'AGN 5',
                              '98SNIa2AGN': 'AGN 2',
                              '99SNIa1AGN': 'AGN 1',
                              #'99.9SNIa0.1AGN': 'AGN 0.1',
                              #'83SNIa17SLSN-I': 'SLSN-I 17',
                              #'90SNIa10SLSN-I': 'SLSN-I 10',
                              #'95SNIa5SLSN-I': 'SLSN-I 5',
                              #'98SNIa2SLSN-I': 'SLSN-I 2',
                              #'99SNIa1SLSN-I': 'SLSN-I 1',
                              #'99SNIa1SLSN': 'SLSN 1',
                              #'99.9SNIa0.1SLSN': 'SLSN-I 0.1',
                              #'95SNIa5TDE': 'TDE 5',
                              #'98SNIa2TDE': 'TDE 2',
                              #'99SNIa1TDE': 'TDE 1',
                              #'99.6SNIa0.4TDE': 'TDE 0.4',
                              '99SNIa1CART': 'CART 1'
                              #'99.1SNIa0.9CART': 'CART 0.9',
                              #'99.7SNIa0.3CART': 'CART 0.3'
                  })
    else:
        remap_dict = OrderedDict({
                          'perfect3000': 'Perfect', 
                          'fiducial3000': 'Fiducial', 
                          'random3000': 'Random',
                          #'all_objs_survived_SALT2_DDF' : 'All SALT',
                          #'all_objs_survived_SALT2_WFD': 'All SALT',
                          #'50SNIa50SNII': 'SN-II 50',
                          #'68SNIa32SNII': 'SN-II 32',
                          '72SNIa28SNII': 'SN-II 28',
                          '75SNIa25SNII': 'SN-II 25', 
                          '90SNIa10SNII': 'SN-II 10',
                          '95SNIa5SNII': 'SN-II 5',
                          '98SNIa2SNII': 'SN-II 2',
                          '99SNIa1SNII': 'SN-II 1',
                          #'50SNIa50SNIbc': 'SN-Ibc 50',
                          #'68SNIa32SNIbc': 'SN-Ibc 32',
                          #'75SNIa25SNIbc': 'SN-Ibc 25',
                          #'83SNIa17SNIbc': 'SN-Ibc 17',
                          #'90SNIa10SNIbc': 'SN-Ibc 10',
                          #'92SNIa8SNIbc': 'SN-Ibc 8',
                          '95SNIa5SNIbc': 'SN-Ibc 5',
                          '98SNIa2SNIbc': 'SN-Ibc 2',
                          '99SNIa1SNIbc': 'SN-Ibc 1',
                          #'50SNIa50SNIax': 'SN-Iax 50',
                          #'68SNIa32SNIax': 'SN-Iax 32',
                          #'75SNIa25SNIax': 'SN-Iax 25',
                          #'86SNIa14SNIax': 'SN-Iax 14',
                          '90SNIa10SNIax': 'SN-Iax 10',
                          #'91SNIa9SNIax': 'SN-Iax 9',
                          #'94SNIa6SNIax': 'SN-Iax 6',
                          '95SNIa5SNIax': 'SN-Iax 5',
                          #'97SNIa3SNIax': 'SN-Iax 3',
                          '98SNIa2SNIax': 'SN-Iax 2',
                          '99SNIa1SNIax': 'SN-Iax 1',
                          #'99.1SNIa0.9CART': 'CART 0.9',
                           '99.4SNIa0.6CART': 'CART 0.6',
                          #'99.7SNIa0.3CART': 'CART 0.3',
                          #'71SNIa29SNIa-91bg': 'SN-Ia-91bg 29',
                          #'75SNIa25SNIa-91bg': 'SN-Ia-91bg 25',
                          #'90SNIa10SNIa-91bg': 'SN-Ia-91bg 10',
                          #'95SNIa5SNIa-91bg': 'SN-Ia-91bg 5',
                          #'98SNIa2SNIa-91bg': 'SN-Ia-91bg 2',
                          #'99SNIa1SNIa-91bg': 'SN-Ia-91bg 1',
                          #'99.8SNIa0.2SNIa-91bg': 'SN-Ia-91bg 0.2',
                          #'57SNIa43AGN': 'AGN 43',
                          #'75SNIa25AGN': 'AGN 25',
                          #'90SNIa10AGN': 'AGN 10',
                          #'94SNIa6AGN': 'AGN 6',
                          #'95SNIa5AGN': 'AGN 5',
                          #'98SNIa2AGN': 'AGN 2',
                          #'99SNIa1AGN': 'AGN 1',
                          #'99.9SNIa0.1AGN': 'AGN 0.1',
                          #'83SNIa17SLSN-I': 'SLSN-I 17',
                          #'90SNIa10SLSN-I': 'SLSN-I 10',
                          #'95SNIa5SLSN-I': 'SLSN-I 5',
                          #'98SNIa2SLSN-I': 'SLSN-I 2',
                          #'99SNIa1SLSN-I': 'SLSN-I 1',
                          #'99SNIa1SLSN': 'SLSN 1',
                          '99.9SNIa0.1SLSN': 'SLSN 0.1',
                          #'95SNIa5TDE': 'TDE 5',
                          #'98SNIa2TDE': 'TDE 2',
                          #'99SNIa1TDE': 'TDE 1',
                          #'99.6SNIa0.4TDE': 'TDE 0.4',
              })
    return(remap_dict)

In [ ]:
remap_dicts = {}
for file_extension in file_extensions:
    thing = make_remap_dict(file_extensions[file_extension])
    tempdict = {}
    for case in cases[file_extension]:
        if case[:-4] in thing.keys():
            tempdict[case[:-4]] = thing[case[:-4]]
        #else:
            #print(case)
    remap_dicts[file_extension] = tempdict#{thing[case[:-4]] for case in cases[file_extension]}

In [ ]:
# Mapping the percent contaminated to the colormap.
## size corresponds to remap_dict
def make_color_nums(file_extension):
    if 'wfd' in file_extension:
        color_num = np.array([1, 1, 1, #1, 1, 1,                    # Special
                           #50, 32, 
                              28, 25, 10, 5, 2, 1,   # II
                           #50, 32, 25, 17, 
                              10, 5, 2, 1,               # Ibc
                           #50, 32, 
                              25, 10, 5, 2, 1,         # Iax
                           #29, 25, 10, 
                              5, 2, 1,                           # 91bg
                           #43, 25, 10, 6, 5, 
                              2, 1,                       # AGN
                           #17, 10, 5, 2, 1, 1, 1,                            # SLSN
                           #5, 2, 1, 1,                            # TDE
                           1                           # CART
                          ]) #+ 1                    
    else:
        color_num = np.array([1, 1, 1, #1, 1, 1,                    # Special
                           #50, 32, 
                              28, 25, 10, 5, 2, 1,   # II
                           #50, 32, 25, 17, 10, 8, 
                              5, 2, 1,               # Ibc
                           #50, 32, 25, 14, 
                              10, 5, 2, 1,         # Iax
                           #1, 
                              1,                           # CART
                           #29, 25, 10, 5, 2, 1, 1,                          # 91bg
                           #43, 25, 10, 6, 5, 2, 1, 1,                      # AGN
                           #17, 10, 5, 2, 1, 1, 
                              1                            # SLSN
                           #5, 2, 1, 1,                            # TDE
                          ]) #+ 1   
    return(color_num)

In [ ]:
color_nums = {}
for file_extension in file_extensions:
    color_nums[file_extension] = make_color_nums(file_extensions[file_extension])

In [ ]:
# Color map
rainbow = cm = plt.get_cmap('plasma_r')
cNorm  = colors.LogNorm(vmin=1, vmax=52) #colors.Normalize(vmin=0, vmax=50)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=rainbow)
color_map = scalarMap.to_rgba(np.arange(1, 52))

## calculate the curve(s)

KDE for each set of posterior samples

In [ ]:
eps = 2. * sys.float_info.min

def safe_log(arr, threshold=eps):
    """
    Takes the natural logarithm of an array that might contain zeros.

    Parameters
    ----------
    arr: ndarray, float
        array of values to be logged
    threshold: float, optional
        small, positive value to replace zeros and negative numbers

    Returns
    -------
    logged: ndarray
        logged values, with small value replacing un-loggable values
    """
    arr = np.asarray(arr)
    arr[arr < threshold] = threshold
    logged = np.log(arr)
    return logged

def make_grid(x, y, x_ngrid=100, y_ngrid=100):
    x_min = x.min()#-1.2
    x_max = x.max()#-0.8
    y_min = y.min()#0.2
    y_max = y.max()#0.4

    x_grid, y_grid = np.mgrid[x_min:x_max:x_ngrid*1.j, y_min:y_max:y_ngrid*1.j]
    x_vec, y_vec = x_grid[:, 0], y_grid[0, :]
    dx = (x_max - x_min) / (x_ngrid - 1)
    dy = (y_max - y_min) / (y_ngrid - 1)

    return(((x_min, y_min), (x_max, y_max)), (x_grid, y_grid), (x_vec, y_vec), (dx, dy))

def make_kde(Xgrid, Ygrid, Xsamps, Ysamps, to_log=False, save=None, one_d=True):
    if not one_d:
        positions = np.vstack([Xgrid.ravel(), Ygrid.ravel()])
        values = np.vstack([Xsamps, Ysamps])
        kernel = sps.gaussian_kde(values, bw_method='scott')
        Z = np.reshape(kernel(positions).T, Xgrid.shape)
    else:
        positions = Xgrid.T[0]
        values = Xsamps
        kernel = sps.gaussian_kde(values, bw_method='scott')
        Z = kernel(positions)
    
    if to_log:
        return safe_log(Z)
    else:
        return Z
#     if save is not None:
# TODO: normalize up here before log!

In [ ]:

# alloutputs = pd.DataFrame(columns=['path', 'KLD'])
#     # make reference sample
# with gzip.open(fullpath+refname) as reffn:
#     flatref = pd.read_csv(reffn)
# [w_ref, Omm_ref] = [flatref['w'], flatref['om']]
# ref_extrema, ref_grids, ref_vecs, ref_ds = make_grid(w_ref, Omm_ref)
# (w_vec, Omm_vec) = ref_vecs
# (dw, dOmm) = ref_ds
# ((xmin, ymin), (xmax, ymax)) = ref_extrema
# (w_grid, Omm_grid) = ref_grids
# d_ref = {'w': dw, 'Omm': dOmm}
# grid_ref = {'w': w_grid, 'Omm': Omm_grid}
# kde_ref = make_kde(w_grid, Omm_grid, w_ref, Omm_ref, one_d=True, to_log=True)

In [ ]:
def get_posteriors(field, k, casename, nsn, withlowz=True):
    
    if 'perfect' in casename or 'random' in casename or 'fiducial' in casename:
        if str(nsn) not in casename:
            case = casename + str(nsn)
        else:
            case = casename
    else:
        case = casename

    filename = 'chains_'+case

    if withlowz:
        filename = filename+'_lowz_withbias'
    path_pre = '/media/RESSPECT/data/PLAsTiCC/for_metrics/final_data3/' + file_extensions[field] + \
               '/results/v' + str(k) + '/' + str(nsn) + '/posteriors/pkl/'
#     if field == 'ddf':
# #         if k == '':
# #             k = 0
#         path_pre = '/media/RESSPECT/data/PLAsTiCC/for_metrics/final_data/DDF/v'+str(k)+'/posteriors/pkl/'
# #             path_pre = '/media/RESSPECT/data/PLAsTiCC/for_metrics/ddf/posteriors/samples_emille/'
# #             ext = '.csv.gz'
# #         else:
# #             path_pre = '/media/RESSPECT/data/PLAsTiCC/for_metrics/ddf/emille_samples'+str(k)+'/posteriors/'
    ext = '.pkl'
#     elif field == 'wfd':
#         path_pre = '/media/RESSPECT/data/PLAsTiCC/for_metrics/wfd/posteriors/samples_emille'+str(k)+'/'
#         ext = '.csv.gz'
    samppathname = path_pre+filename+ext

    if ext == '.csv.gz':
        with gzip.open(samppathname) as sampfile:
            sampdata = pd.read_csv(sampfile)
    elif ext == '.pkl':
        with open(samppathname, 'rb') as sampfile:
            sampdata = pkl.load(sampfile)
#     print(sampdata)
    return([sampdata['w'], sampdata['om']])

In [ ]:
null_cases = ['perfect', 'random', 'fiducial']
ktot = 1
kmin = 0
samp_sizes = [1500, 3000, 6000]
ngrid = 100

In [ ]:
outdata = {}
for field in file_extensions:
    outdata[field] = {}
    for casename in null_cases:
        outdata[field][casename] = np.empty((ktot, len(samp_sizes), 2, ngrid))
        for k in range(kmin, ktot, 1):
            for i, nsn in enumerate(samp_sizes):
                kpass = k
                [w_comp, Omm_comp] = get_posteriors(field, kpass, casename, nsn, withlowz=True)#[sampdata['w'], sampdata['om']]
                comp_extrema, comp_grids, comp_vecs, comp_ds = make_grid(w_comp, Omm_comp)
                (w_grid, Omm_grid) = comp_grids
                kde_comp = make_kde(w_grid, Omm_grid, w_comp, Omm_comp, one_d=True, to_log=True)
                outdata[field][casename][k][i] = np.array([w_grid.T[0], kde_comp])
with open('default_kdes.pkl', 'wb') as outfile:
    pkl.dump(outdata, outfile)

In [ ]:
outdata = {}
for field in file_extensions:
    outdata[field] = {}
    for casename in cases[field]:
        outdata[field][casename[:-4]] = np.empty((2, ngrid))
        k = '0'
        nsn = '3000'
        [w_comp, Omm_comp] = get_posteriors(field, k, casename[:-4], nsn, withlowz=True)#[sampdata['w'], sampdata['om']]
        comp_extrema, comp_grids, comp_vecs, comp_ds = make_grid(w_comp, Omm_comp)
        (w_grid, Omm_grid) = comp_grids
        kde_comp = make_kde(w_grid, Omm_grid, w_comp, Omm_comp, one_d=True, to_log=True)
        outdata[field][casename[:-4]] = np.array([w_grid.T[0], kde_comp])
with open('testcase_kdes.pkl', 'wb') as outfile:
    pkl.dump(outdata, outfile)

## make plot(s)

In [ ]:
def_colors = {'perfect': 'k', 'random': 'tab:red', 'fiducial': 'tab:blue'}
def_styles = {'1500': ':', '3000': '-', '6000': '--'}#{'DDF': '-', 'WFD': '--'}
# def_lowz = {'withbias': , 'nobias':}

with open('default_kdes.pkl', 'rb') as infile:
    indata = pkl.load(infile)

fig, ax = plt.subplots(2, 1, figsize=(6, 7))    
for j, field in enumerate(file_extensions):
    for casename in null_cases:
        ax[j].scatter([0], [0], label=casename, color=def_colors[casename])
        for i, nsn in enumerate(samp_sizes):
            for k in range(ktot-1, ktot):
                w_grid, kde_comp = indata[field][casename][k][i]#[w_grid, kde_comp] = indata[casename]
#                 if k == 0:
#                     lw_boost = 2
# #                     print(kde_comp)
#                 else:
#                     lw_boost = 1
                ax[j].plot(w_grid, np.exp(kde_comp),# label=field+casename, 
                linestyle=def_styles[str(nsn)], color=def_colors[casename], alpha=0.8, linewidth=1.25)
    for nsn in samp_sizes:
        ax[j].plot([0], [0], label=str(nsn), 
                 linestyle=def_styles[str(nsn)], color='tab:green', alpha=1., linewidth=1.25) 
    ax[j].set_yticks([10, 30, 50])
    ax[j].set_yticklabels([10, 30, 50], fontsize=14)
    ax[j].set_ylabel(r'PDF ($w^{-1}$)', fontsize=18)
    ax[j].vlines(-1., ax[j].get_ylim()[0], ax[j].get_ylim()[1], color='gray', alpha=0.5)
    #ax[j].set_ylim(0., 70.)
    if j == 0:
        yset = ax[j].get_ylim()[1]
        ax[j].text(-1.175, 0.85*yset, file_extensions[field], fontsize=20)
        ax[j].set_xticks([])
    # plt.title(field+k)
    if j == 1:
        ax[j].set_xticks([-1.2, -1.1, -1.])
        ax[j].set_xticklabels([-1.2, -1.1, -1.], fontsize=14)
        ax[j].legend(loc='lower left', fontsize=14)#, ncol=2)
        ax[j].set_xlabel(r'$w$', fontsize=18)
        yset = ax[j].get_ylim()[1]
        ax[j].text(-1.175, 0.85*yset, file_extensions[field], fontsize=20)
    ax[j].set_xlim(-1.2, -0.95)
fig.subplots_adjust(wspace=0., hspace=0.)
plt.savefig('dists_null.png', bbox_inches='tight' ,dpi=250)
#plt.show()

todo: investigate the runs that are flat KDEs

todo: also with and without bias of lowz sample

In [ ]:
rates, contaminants = {}, {}
for field in file_extensions:
    rate, contaminant = {}, {}
    for key in remap_dicts[field]:
        postsplit = remap_dicts[field][key].split()
        if len(postsplit) > 1:
            name = postsplit[0]
            perc = float(postsplit[-1])
#         rate[name] = perc
            rate[key] = perc
            contaminant[key] = name
    rates[field] = rate
    contaminants[field] = contaminant

In [ ]:
# for field in file_extensions:
#     plt.hist(rates[field].values(), bins=25, alpha=0.5, label=field)
# plt.legend()

todo: automate dividing into panels

In [ ]:
cutoffs = [0., 1., 2., 5., 7.5, 15., 50.]
cutofflabels = ['<1%', '1%', '2%', '5%', '10%', '25%']

panel_groups = {}
for field in file_extensions:
    panel_groups[field] = {j: [] for j in range(6)}
#     print(field)
    for i, casefn in enumerate(rates[field]):
        casename = casefn#[:-4]
        rate = rates[field][casename]
#         for j, cutoff in enumerate(cutoffs[:-1]):
#             if rate > cutoffs[j] and rate < :
#                 panel_groups[field][0].append(casename)
        if rate > 0. and rate < 1.:
            panel_groups[field][0].append(casename)
#             print((casename, rates[field][casename], 0))
        elif rate >= 1. and rate < 2.:
            panel_groups[field][1].append(casename)
#             print((casename, rates[field][casename], 1))
        elif rate >= 2. and rate < 5.:
            panel_groups[field][2].append(casename)
#             print((casename, rates[field][casename], 1))
        elif rate >= 5. and rate < 7.5:
            panel_groups[field][3].append(casename)
#             print((casename, rates[field][casename], 5))
        elif rate >= 7.5 and rate <= 15.:
            panel_groups[field][4].append(casename)
#             print((casename, rates[field][casename], 10))  
        elif rate >= 15. and rate <= 50.:
            panel_groups[field][5].append(casename)
#             print((casename, rates[field][casename], 25))    
#         else:
#             print((casename, rates[field][casename], 'big'))
print(panel_groups)

In [ ]:
ddf_set = set(contaminants['ddf'].values())
# if len(file_extensions) > 1:
#     for field in file_extensions[1:]:
#         base_contaminant_set = set.union(base_contaminant_set, set(contaminants[field].values()))
wfd_set = set(contaminants['wfd'].values())
all_contaminants = set.union(ddf_set, wfd_set)
# base_contaminant_set#

color_list = OrderedDict({contaminant: plt.cm.tab10(i) for i, contaminant in enumerate(all_contaminants)})

contaminant_colors = {}
for field in file_extensions:
    contaminant_colors[field] = {}
    for i, contaminant in enumerate(contaminants[field]):
        contaminant_colors[field][contaminant] = color_list[contaminants[field][contaminant]]

In [ ]:
ax.get_yticks()

In [ ]:
# def_colors = {'perfect': 'k', 'random': 'tab:red', 'fiducial': 'tab:blue'}
# def_styles = {'1500': ':', '3000': '-', '6000': '--'}#{'DDF': '-', 'WFD': '--'}
# def_lowz = {'withbias': , 'nobias':}

axs = {}

with open('testcase_kdes.pkl', 'rb') as infile:
    indata = pkl.load(infile)

for field in file_extensions:
    table_loc = '/media/RESSPECT/data/PLAsTiCC/for_metrics/final_data/'+file_extensions[field]+'/v'+str(0)+'/summary_stats.csv'
#     if field == 'wfd':
#         table_loc = '/media2/RESSPECT2/data/posteriors_wfd/omprior_0.01_flat/summary_cases_omprior_0.01_flat_emille.csv'
#     else:
#         table_loc = '/media2/RESSPECT2/data/posteriors_ddf/omprior_0.01_flat/summary_cases_emille.csv'
    df = pd.read_csv(table_loc)
    df = df.set_index('case')
#     print((field, df.index))
    fig = pylab.figure(figsize=(15, 10))
    bigAxes = pylab.axes(frameon=False)     # hide frame
    bigAxes.set_xticks([])                        # don't want to see any ticks on this axis
    bigAxes.set_yticks([])
#     bigAxes.set_xlabel(r'$-1.3 < w < -0.9$', fontsize=20)
    bigAxes.set_title(file_extensions[field], fontsize=20)
    numrows=2
    numcols=3
#     fig, ax = plt.subplots(2, 3, figsize=(15, 10))
#     fig.suptitle(file_extensions[field])
    for i in range(len(panel_groups[field])):
        per_panel_contaminants = [contaminants[field][panel_groups[field][i][j]] 
                                  for j in range(len(panel_groups[field][i]))]
        uniques, unique_ind = np.unique(per_panel_contaminants, return_index=True)
        
        axs[i] = fig.add_subplot(numrows,numcols,i+1)
        ax = axs[i]
#         ax.spines['right'].set_visible(False)
#         ax.spines['top'].set_visible(False)
#         position = ax.get_position()
#         position.x0 += 0.01
#         position.y0 += 0.02
#         position.x1 += 0.01
#         position.y1 += 0.02
#         ax.set_position(position)
        stylecount = 0
        for j, val in enumerate(unique_ind):
            casename = panel_groups[field][i][val]
            w_grid, kde_comp = indata[field][casename]#[w_grid, kde_comp] = indata[casename]
            ax.plot(w_grid, np.exp(kde_comp), color=contaminant_colors[field][casename], label=per_panel_contaminants[val]) 
#         plt.title(field)
#             ax.vlines(df['wfit_w_lowz'].loc[casename], 0, 40, color=contaminant_colors[field][casename], linestyle='--')
#             print(df['wfit_w_lowz'].loc[casename])
       
        l = ax.legend(fontsize=16, loc='upper left', bbox_to_anchor=(-0.025, 1.025), title=cutofflabels[i])#, ncol=2)
        plt.setp(l.get_title(),fontsize=14)
        ax.set_xlim(-1.25, -0.95)
        #ax.text(-1.175, ax.get_ylim()[1]*0.15, cutofflabels[i], fontsize=18, horizontalalignment='right', verticalalignment='center')
        if i < 3 and field == 'ddf':
            axs[i].set_ylim(0., 30.)
            #ax.set_yticklabels([0, 10, 20, 30], fontsize=16)
        elif i > 2 and field == 'ddf':
            axs[i].set_ylim(0, 45)
            
        if i < 3 and field == 'wfd':
            axs[i].set_ylim(0., 25.)
            #ax.set_yticklabels([0, 10, 20, 30], fontsize=16)
        elif i > 2 and field == 'wfd':
            axs[i].set_ylim(0, 30)
            
        if i%3 == 0:
            ax.set_ylabel(r'PDF ($w^{-1}$)', fontsize=18)
            #ax.set_yticks(ax.get_yticks())
            #ax.set_yticklabels(ax.get_yticks(), fontsize=16)

            #ax.yaxis.set_major_locator(ax.get_yticks(), MaxNLocator(integer=True))
        elif i in [1,2]:
            ax.set_ylim(axs[0].get_ylim())
        elif i in [4,5]:
            ax.set_ylim(axs[3].get_ylim())
        ax.set_xlabel(r'$w$', fontsize=18)
        ax.set_xticks([-1.2, -1.1, -1.])
        ax.set_xticklabels([-1.2, -1.1, -1.], fontsize=16)
        ax.vlines(-1., ax.get_ylim()[0], ax.get_ylim()[1], color='gray', alpha=0.5)
        ax.yaxis.set_major_locator(MaxNLocator(integer=True))
#     lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
#     lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
#     fig.legend(lines, labels)
#         ax.set_xticklabels([])
#     plt.savefig(file_extensions[field]+'dists_null.png')

    for ii in [1,2,4,5]:
        axs[ii].set_yticks([])
        
    axs[0].set_yticks([0, 10, 20, 30])
    axs[0].set_yticklabels([0, 10, 20, 30], fontsize=14)

    if field == 'ddf':    
        axs[3].set_yticks([0, 10, 20, 30,40])
        axs[3].set_yticklabels([0, 10, 20, 30,40], fontsize=14)
    else:    
        axs[3].set_yticks([0, 5, 15, 25])
        axs[3].set_yticklabels([0, 5, 15, 25], fontsize=14)
    
    fig.subplots_adjust(wspace=0., hspace=0.)
    pylab.savefig(file_extensions[field]+'combos.png',format='png', bbox_inches='tight', pad_inches=0.3, dpi=250)
    #fig.show()

TODO: polish these for paper
- linestyle for contamination rate if more than one with same contaminant per panel